In [108]:
import psycopg2
import pandas as pd
from faker import Faker
from faker_music import MusicProvider
from faker.providers.date_time import Provider as DatetimeProvider
from random import randrange, choice
import datetime
faker = Faker()
conn = psycopg2.connect("host=db dbname=postgres user=postgres password=root")
faker.add_provider(MusicProvider)
faker.add_provider(DatetimeProvider)
    
def execute_commands(commands, printflag=None):
    if isinstance(commands, str): commands = [commands]
    
    for command in commands:
        try:
            cur = conn.cursor()
            cur.execute(command)
            if printflag: print(cur.fetchall())
            cur.close()
        except Exception as error:
            print(error)
            conn.rollback()
            cur.close()
            cur = None

    # close communication with the PostgreSQL database server
    # commit the changes
    conn.commit()


In [5]:
def sql_to_dataframe(conn, query, column_names):
    """Import data from a PostgreSQL database using a SELECT query """
    cursor = conn.cursor()
    try:
        cursor.execute(query)
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        cursor.close()
        return 1
    # The execute returns a list of tuples:
    tuples_list = cursor.fetchall()
    cursor.close()
    # Now we need to transform the list into a pandas DataFrame:
    df = pd.DataFrame(tuples_list, columns=column_names)
    return df

In [38]:
# Drop all tables
delete_table_commands = [
"DROP SCHEMA public CASCADE;",
"CREATE SCHEMA public;"
]
execute_commands(delete_table_commands)
faker.unique.clear()

In [39]:
# Create all tables
create_table_commands = [
    """
    CREATE TABLE persons (
        person_id SERIAL PRIMARY KEY,
        first_name VARCHAR(255) NOT NULL,
        last_name VARCHAR(255) NOT NULL
    )
    """,
    """
    CREATE TABLE albums (
      album_id SERIAL PRIMARY KEY
      , title VARCHAR(255) NOT NULL
      , author VARCHAR(255) NOT NULL 
    )
    """,
    """
    CREATE TABLE copies(
      copy_id SERIAL PRIMARY KEY
      , album_id INTEGER NOT NULL REFERENCES albums (album_id)
      , person_id INTEGER NOT NULL REFERENCES persons (person_id)
    )
    """,
    """
    CREATE TABLE loans(
      loan_id SERIAL PRIMARY KEY
      , copy_id INTEGER NOT NULL REFERENCES copies (copy_id)
      , start_date DATE NOT NULL 
      , end_date DATE NOT NULL
      , person_id INTEGER NOT NULL REFERENCES persons (person_id)
    )
    """
]

execute_commands(create_table_commands)



In [40]:
# Generate fake persons
fake_person_commands = []
person_data = [(faker.unique.random_int(), faker.first_name(),faker.last_name() ) for _ in range(10)]

In [41]:
# Insert fake people
for data in person_data:
    fake_person_commands.append(f"INSERT INTO persons(person_id, first_name, last_name) VALUES ({data[0]}, '{data[1]}', '{data[2]}')")
    
execute_commands(fake_person_commands)

In [42]:
# Get dataframe from person table
column_names = ['person_id','first_name','last_name']
query = """SELECT * FROM persons;"""
persons_df = sql_to_dataframe(conn, query, column_names)
print(persons_df)

   person_id first_name last_name
0        238     Joanna  Bautista
1       9455        Kim      Lane
2       4636      Jesse    George
3       6800      Wendy   Johnson
4       1712    Rachael     Smith
5       2736        Kim  Petersen
6        554     Jeremy  Thompson
7       2277       Erin    Suarez
8       9799     Kelsey   Jackson
9       4082     Regina   Roberts


In [43]:
# Generate fake albums
fake_album_commands = []
album_data = [(faker.unique.random_int(), faker.music_subgenre(), faker.first_name()+ " " + faker.last_name()) for _ in range(7)]

In [44]:
# Insert fake albums
for data in album_data:
    fake_album_commands.append(f"INSERT INTO albums(album_id, title, author) VALUES ({data[0]}, '{data[1]}', '{data[2]}')")
    
execute_commands(fake_album_commands)

In [45]:
# Get dataframe from album table
column_names = ['album_id','title','author']
query = """SELECT * FROM albums;"""
albums_df = sql_to_dataframe(conn, query, column_names)
print(albums_df)

   album_id                title             author
0      2411                March        Jack Knight
1      9579               Boogie     Kathryn Martin
2      1885     Contemporary R&B        James Woods
3       762                March     Madison Cortez
4      6366  Contemporary Celtic      April Cabrera
5      4852         Parody Music  Matthew Armstrong
6      8886          Iranian Pop         Erin Gibbs


In [46]:
# Generate fake copies
fake_copy_commands = []
copy_data = [(faker.unique.random_int(), albums_df.iloc[randrange(albums_df.shape[0])]['album_id'], persons_df.iloc[randrange(persons_df.shape[0])]['person_id']) for i in range(1,10)]

# Insert fake copies
for data in copy_data:
    fake_copy_commands.append(f"INSERT INTO copies(copy_id, album_id, person_id) VALUES ({data[0]}, '{data[1]}', '{data[2]}')")

execute_commands(fake_copy_commands)

In [47]:
# Get dataframe from copies table
column_names = ['copy_id','album_id','person_id']
query = """SELECT * FROM copies;"""
copies_df = sql_to_dataframe(conn, query, column_names)
print(copies_df)

   copy_id  album_id  person_id
0     2773      8886       4636
1      913      8886       2736
2      612      6366       4636
3     1371      2411       4082
4     1080      9579       6800
5     6915       762       4636
6     5871      6366        238
7     8248      8886       9455
8     5396      2411       6800


In [86]:
# Generate fake loans
loan_data = []
# Current loans interval
earliest_start_loan_date = datetime.date.fromisoformat('2022-12-04')
latest_end_loan_date = datetime.date.fromisoformat('2023-06-01')

copy_id_list = copies_df['copy_id'].tolist()
person_id_list = persons_df['person_id'].tolist()
for copy_id in copy_id_list:  # for all copies
    # Generate 0-3 non-overlapping past loans for each copy
    num_loans = randrange(0, 3)
    first_available_date = datetime.date.today() - datetime.timedelta(days=365)  # start loaning out this copy from a year ago
    for _ in range(num_loans):
        loan_id = faker.unique.random_int()
        start_date = faker.date_between(first_available_date)
        end_date = faker.date_between(start_date)
        person_id = choice(person_id_list)
        loan_data.append((loan_id, copy_id, start_date, end_date,person_id))

        first_available_date = end_date  # update first_available_date to avoid overlapping loans for the same copy


In [114]:
fake_loans_commands = []

for data in loan_data:
    # date formatting requires to not use execute_commands
    try:
        cur = conn.cursor()
        cur.execute("INSERT INTO loans(loan_id, copy_id, start_date, end_date, person_id) VALUES (%s, %s, %s, %s, %s)", (data[0], data[1], data[2], data[3], data[4]))
        cur.close()
    except Exception as error:
        print(error)
        conn.rollback()
        cur.close()
        cur = None
    conn.commit()

duplicate key value violates unique constraint "loans_pkey"
DETAIL:  Key (loan_id)=(4427) already exists.

duplicate key value violates unique constraint "loans_pkey"
DETAIL:  Key (loan_id)=(1243) already exists.

duplicate key value violates unique constraint "loans_pkey"
DETAIL:  Key (loan_id)=(63) already exists.

duplicate key value violates unique constraint "loans_pkey"
DETAIL:  Key (loan_id)=(2049) already exists.

duplicate key value violates unique constraint "loans_pkey"
DETAIL:  Key (loan_id)=(2950) already exists.

duplicate key value violates unique constraint "loans_pkey"
DETAIL:  Key (loan_id)=(8615) already exists.

duplicate key value violates unique constraint "loans_pkey"
DETAIL:  Key (loan_id)=(6549) already exists.

duplicate key value violates unique constraint "loans_pkey"
DETAIL:  Key (loan_id)=(4397) already exists.

duplicate key value violates unique constraint "loans_pkey"
DETAIL:  Key (loan_id)=(692) already exists.

duplicate key value violates unique cons

In [109]:
OWNER_ID = 238
execute_commands(f"SELECT * FROM copies WHERE person_id = {OWNER_ID};", 1)

[(5871, 6366, 238)]


In [111]:
execute_commands("SELECT copy_id, COUNT(*) FROM loans GROUP BY copy_id HAVING COUNT(*) > 1;",1)

[(5871, 2), (913, 2), (6915, 2), (612, 2)]


In [112]:
execute_commands("SELECT * FROM loans WHERE end_date - start_date > 7;",1)

[(4427, 2773, datetime.date(2022, 9, 10), datetime.date(2022, 11, 27), 9455), (1243, 913, datetime.date(2022, 10, 13), datetime.date(2023, 4, 25), 9455), (2049, 612, datetime.date(2022, 7, 4), datetime.date(2022, 11, 12), 9455), (2950, 612, datetime.date(2023, 1, 29), datetime.date(2023, 3, 9), 6800), (8615, 1371, datetime.date(2022, 6, 3), datetime.date(2022, 11, 10), 4636), (6549, 1080, datetime.date(2022, 12, 2), datetime.date(2023, 4, 16), 554), (6812, 5396, datetime.date(2022, 8, 13), datetime.date(2022, 12, 28), 9799)]


In [113]:
execute_commands("SELECT * FROM albums WHERE album_id NOT IN (SELECT DISTINCT album_id FROM copies JOIN loans ON copies.copy_id = loans.copy_id);",1)

[(1885, 'Contemporary R&B', 'James Woods'), (4852, 'Parody Music', 'Matthew Armstrong')]
